<a href="https://colab.research.google.com/github/ltpdo/GradeData-MachineLearing/blob/formulas/%E7%A7%91%E7%9B%AE%E3%81%94%E3%81%A8%E3%81%AE%E6%88%90%E7%B8%BE%E7%AE%97%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ライブラリのインポート

In [3]:
!pip install gradio
!pip install gspread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.7 MB/s eta 

In [4]:
# ライブラリのインポート
import gradio as gr
import gspread
import pandas as pd
import numpy as np

In [5]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [6]:
# Google Driveとの接続
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/卒論/成績予測する科目/')

Mounted at /content/drive


# データの前処理

In [ ]:
# グーグルスプレッドシートをurlで開く
url = "https://docs.google.com/spreadsheets/d/1u0DKe-96bHj64VYDR3JPBT2m610ASQhgCzyZxDc1c0w/edit#gid=64615062"
workbook = gc.open_by_url(url)

# シートの特定を行う
sheet = workbook.worksheet("科目ごとの回帰係数(削除版)")

In [ ]:
# スプレッドシートのデータをPandasのDataframeに変換する。
rows = sheet.get_all_values()
df = pd.DataFrame.from_records(rows)
df

In [ ]:
# データフレームを変換
grade_sheet = df.T
grade_sheet.columns = grade_sheet.iloc[0]
grade_sheet = grade_sheet.iloc[1:]

# 空白をNaNに変換
grade_sheet.replace('', np.nan, inplace=True)

# 使用しない科目を削除
grade_sheet = grade_sheet.dropna(subset = ["test_size", "test"])

# 列を削除
grade_sheet = grade_sheet.dropna(axis=1, how='all')
grade_sheet = grade_sheet.drop(columns = ["test_size", "test"])

# indexに指定
grade_sheet.set_index(grade_sheet.iloc[:, 0], inplace = True)
grade_sheet.drop(columns = grade_sheet.columns[0], inplace = True)

# 欠損値を補完
grade_sheet =  grade_sheet.fillna(0)

# データの表示
grade_sheet

# データの保存
grade_sheet.to_csv('/content/drive/MyDrive/Colab Notebooks/卒論/成績予測する科目/方程式に使うデータフレーム.csv')

# データの数式化


In [ ]:
# grade_sheet DataFrame を辞書型に変換します
data_dict = grade_sheet.to_dict(orient='index')

print(data_dict)

In [ ]:
# 辞書型を変換する関数
def convert_regression_coefficients(data, prefix='回帰係数', start_index=1):
    """
    回帰係数のデータを変換する関数。

    Args:
        data (dict): 変換するデータの辞書。
        prefix (str, optional): 回帰係数のキーのプレフィックス。デフォルトは'回帰係数'。
        start_index (int, optional): 回帰係数の数値の開始インデックス。デフォルトは1。

    Returns:
        dict: 変換されたデータの辞書。
    """
    # 変換されたデータを格納するための空の辞書を作成
    converted_data = {}

    # 元のデータの各要素に対して処理を行う
    for key, value in data.items():
        # 変換された回帰係数を格納するための空の辞書を作成
        converted_coefficients = {}

        # 切片を格納
        if '切片' in value:
            converted_coefficients['切片'] = float(value['切片'])

        # 各要素のキーに対して処理を行う
        for coefficient_key in value.keys():
            # 指定されたプレフィックスで始まるキーの場合
            if coefficient_key.startswith(prefix):
                # 対応する値を取得
                coefficient_value = value[coefficient_key]

                # 値が文字列であることを確認
                if isinstance(coefficient_value, str):
                    # 文字列を空白文字で分割し、科目名と係数に分ける
                    subject_key, coefficient_value = coefficient_value.split()

                    # 科目名をキーとして、係数を値として格納
                    converted_coefficients[subject_key] = float(coefficient_value)

        # 元のデータのキーに対応付け、変換された回帰係数を格納
        converted_data[key] = converted_coefficients

    # 変換されたデータを返す
    return converted_data



# 回帰係数のキーを科目名に変更
converted_data = convert_regression_coefficients(data_dict)

print(converted_data)

In [ ]:
def generate_formula(data):
    """
    データから数式を生成する関数。

    Args:
        data (dict): 数式を生成するためのデータ。

    Returns:
        dict: 各データの数式を格納した辞書。
    """
    formulas = {}

    for category, values in data.items():
        formula = ""
        if '切片' in values:
            formula += f"{values['切片']}"

        for key, value in values.items():
            if key != '切片':
                if value != 0:
                    formula += f" + {value} * {key}"

        formula = formula.lstrip(' +')
        formulas[category] = formula

    return formulas

# 与えられたデータ
formulas = generate_formula(converted_data)
print(formulas)

# Gradioの下準備

In [ ]:
# 変数名と日本語の科目名の対応辞書
subject_mapping = {
    "basic_physics": "基礎物理",
    "information_theory": "情報理論・確率論",
    "computer_architecture_II": "計算機構成論Ⅱ",
    "hardware_design_I": "ハードウェア設計Ⅰ",
    "career_development": "キャリア形成基礎論",
    "calculus_II": "微分積分Ⅱ",
    "linear_algebra_II": "線形代数Ⅱ",
    "statistics": "統計学",
    "AI_programming": "ＡＩプログラミング演習",
    "japanese_expression": "日本語表現法",
    "programming_basic_II": "プログラミング基礎Ⅱ",
    "information_science_exercise": "情報科学基礎演習",
    "basic_math": "基礎数学",
    "discrete_math_I": "離散数学Ⅰ",
    "sports_science_exercise": "スポーツ科学演習",
    "information_literacy": "情報リテラシー",
    "programming_basic_I": "プログラミング基礎Ⅰ",
    "mobile_programming_exercise": "モバイルプログラミング演習",
    "mathematical_culture_I": "数理的教養Ⅰ",
    "embedded_software_exercise": "組込みソフトウェア演習",
    "computer_architecture_I": "計算機構成論Ⅰ",
    "programming_languages_and_compilers": "プログラミング言語とコンパイラ",
    "linear_algebra_I": "線形代数Ⅰ",
    "discrete_math_III": "離散数学Ⅲ",
    "programming_intro": "プログラミング入門",
    "discrete_math_II": "離散数学Ⅱ",
    "calculus_I": "微分積分Ⅰ",
    "basic_seminar": "基礎ゼミナール"
}

In [ ]:
def subject_formulas(input_data):
  # 値の格納
  computer_network = calculate_computer_network(input_data)
  data_structures_and_algorithms_I = calculate_data_structures_and_algorithms_I(input_data)
  data_structures_and_algorithms_I_exercise = calculate_data_structures_and_algorithms_I_exercise(input_data)
  hardware_experiment_I = calculate_hardware_experiment_I(input_data)
  hardware_design_I = calculate_hardware_design_I(input_data)
  hardware_design_II = calculate_hardware_design_II(input_data)
  programming_basic_I = calculate_programming_basic_I(input_data)
  programming_basic_II = calculate_programming_basic_II(input_data)
  mobile_programming_exercise = calculate_mobile_programming_exercise(input_data)
  basic_physics = calculate_basic_physics(input_data)
  calculus_I = calculate_calculus_I(input_data)
  calculus_II = calculate_calculus_II(input_data)
  mathematical_culture_I = calculate_mathematical_culture_I(input_data)
  linear_algebra_II = calculate_linear_algebra_II(input_data)
  computer_architecture_I = calculate_computer_architecture_I(input_data)
  computer_architecture_II = calculate_computer_architecture_II(input_data)
  discrete_math_II = calculate_discrete_math_II(input_data)
  ai_programming_exercise = calculate_ai_programming_exercise(input_data)
  information_theory_and_probability = calculate_information_theory_and_probability(input_data)

  # 関数の定義
  # コンピュータネットワーク
  def calculate_computer_network(input_data):
    return (
        -0.098 +
        0.311357 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.30319 * input_data.get(subject_mapping["information_theory"], 0) +
        0.236033 * input_data.get(subject_mapping["computer_architecture_II"], 0) +
        0.157423 * input_data.get(subject_mapping["hardware_design_I"], 0) +
        0.119175 * input_data.get(subject_mapping["career_development"], 0) +
        0.117778 * input_data.get(subject_mapping["calculus_II"], 0) +
        0.113565 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.10379 * input_data.get(subject_mapping["statistics"], 0) +
        0.095154 * input_data.get(subject_mapping["AI_programming"], 0) +
        0.092831 * input_data.get(subject_mapping["japanese_expression"], 0) +
        0.079273 * input_data.get(subject_mapping["programming_basic_II"], 0) +
        0.073856 * input_data.get(subject_mapping["information_science_exercise"], 0) +
        0.071869 * input_data.get(subject_mapping["basic_math"], 0) +
        0.065405 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.064598 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.052042 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.051477 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.036605 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.034946 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.034307 * input_data.get(subject_mapping["embedded_software_exercise"], 0) +
        0.033903 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.028462 * input_data.get(subject_mapping["programming_languages_and_compilers"], 0) +
        0.028391 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.027924 * input_data.get(subject_mapping["discrete_math_III"], 0) +
        0.027779 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.025641 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.017432 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.009304 * input_data.get(subject_mapping["basic_seminar"], 0)
    )

  # データ構造とアルゴリズムⅠ
  def calculate_data_structures_and_algorithms_I(input_data):
    return (
        -0.116 +
        0.249704 * input_data.get(subject_mapping["computer_architecture_II"], 0) +
        0.221179 * input_data.get(subject_mapping["AI_programming"], 0) +
        0.195909 * input_data.get(subject_mapping["japanese_expression"], 0) +
        0.164167 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.160515 * input_data.get(subject_mapping["basic_math"], 0) +
        0.157134 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.122502 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.100162 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.099507 * input_data.get(subject_mapping["statistics"], 0) +
        0.096101 * input_data.get(subject_mapping["hardware_design_I"], 0) +
        0.093628 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.079364 * input_data.get(subject_mapping["career_development"], 0) +
        0.07119 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.064112 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.060764 * input_data.get(subject_mapping["calculus_II"], 0) +
        0.047875 * input_data.get(subject_mapping["programming_languages_and_compilers"], 0) +
        0.042242 * input_data.get(subject_mapping["discrete_math_III"], 0) +
        0.037724 * input_data.get(subject_mapping["embedded_software_exercise"], 0) +
        0.026553 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.022352 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.017834 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.015378 * input_data.get(subject_mapping["information_science_exercise"], 0) +
        0.013546 * input_data.get(subject_mapping["programming_basic_II"], 0) +
        0.009441 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.007062 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.003975 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.003894 * input_data.get(subject_mapping["information_theory"], 0) +
        0.003214 * input_data.get(subject_mapping["mobile_programming_exercise"], 0)
    )

  # データ構造とアルゴリズムⅠ演習
  def calculate_data_structures_and_algorithms_I_exercise(input_data):
    return (
        -0.195 +
        0.422306 * input_data.get(subject_mapping["embedded_software_exercise"], 0) +
        0.392583 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.360221 * input_data.get(subject_mapping["programming_basic_II"], 0) +
        0.338522 * input_data.get(subject_mapping["japanese_expression"], 0) +
        0.337622 * input_data.get(subject_mapping["basic_math"], 0) +
        0.32016 * input_data.get(subject_mapping["AI_programming"], 0) +
        0.280516 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.262953 * input_data.get(subject_mapping["hardware_design_I"], 0) +
        0.250562 * input_data.get(subject_mapping["discrete_math_III"], 0) +
        0.250428 * input_data.get(subject_mapping["information_science_exercise"], 0) +
        0.23222 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.227423 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.211723 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.152606 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.13896 * input_data.get(subject_mapping["calculus_II"], 0) +
        0.132966 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.129423 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.125059 * input_data.get(subject_mapping["programming_languages_and_compilers"], 0) +
        0.114643 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.111073 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.105529 * input_data.get(subject_mapping["information_theory"], 0) +
        0.076624 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.056561 * input_data.get(subject_mapping["statistics"], 0) +
        0.051513 * input_data.get(subject_mapping["career_development"], 0) +
        0.042155 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.03695 * input_data.get(subject_mapping["computer_architecture_II"], 0) +
        0.017071 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.008668 * input_data.get(subject_mapping["basic_seminar"], 0)
    )

  # ハードウェア実験Ⅰ
  def calculate_hardware_experiment_I(input_data):
    return (
        -0.527 +
        0.718769 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.613128 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.41083 * input_data.get(subject_mapping["AI_programming"], 0) +
        0.376113 * input_data.get(subject_mapping["embedded_software_exercise"], 0) +
        0.375078 * input_data.get(subject_mapping["basic_math"], 0) +
        0.349367 * input_data.get(subject_mapping["career_development"], 0) +
        0.316182 * input_data.get(subject_mapping["hardware_design_I"], 0) +
        0.266639 * input_data.get(subject_mapping["programming_basic_II"], 0) +
        0.253246 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.232929 * input_data.get(subject_mapping["computer_architecture_II"], 0) +
        0.227067 * input_data.get(subject_mapping["information_science_exercise"], 0) +
        0.222597 * input_data.get(subject_mapping["statistics"], 0) +
        0.217493 * input_data.get(subject_mapping["programming_languages_and_compilers"], 0) +
        0.214766 * input_data.get(subject_mapping["japanese_expression"], 0) +
        0.200759 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.180837 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.177103 * input_data.get(subject_mapping["information_theory"], 0) +
        0.152963 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.151918 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.148662 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.145688 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.137602 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.13022 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.120696 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.09957 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.075057 * input_data.get(subject_mapping["calculus_II"], 0) +
        0.064852 * input_data.get(subject_mapping["discrete_math_III"], 0) +
        0.026243 * input_data.get(subject_mapping["linear_algebra_I"], 0)
    )

  # ハードウェア設計Ⅰ
  def calculate_hardware_design_I(input_data):
    return (
        0.078 +
        0.192538 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.109622 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.108994 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.096876 * input_data.get(subject_mapping["career_development"], 0) +
        0.085246 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.075823 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.071784 * input_data.get(subject_mapping["information_science_exercise"], 0) +
        0.071515 * input_data.get(subject_mapping["basic_math"], 0) +
        0.069251 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.055008 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.052309 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.030762 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.029598 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.02272 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.016723 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.012811 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.002573 * input_data.get(subject_mapping["sports_science_exercise"], 0)
    )

  # ハードウェア設計Ⅱ
  def calculate_hardware_design_II(input_data):
    return (
        0.206 +
        0.230415 * input_data.get(subject_mapping["computer_architecture_II"], 0) +
        0.187705 * input_data.get(subject_mapping["information_theory"], 0) +
        0.175978 * input_data.get(subject_mapping["hardware_design_I"], 0) +
        0.154934 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.142836 * input_data.get(subject_mapping["programming_basic_II"], 0) +
        0.108172 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.105155 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.100465 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.098126 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.093813 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.086095 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.06779 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.060718 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.055768 * input_data.get(subject_mapping["japanese_expression"], 0) +
        0.045959 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.043688 * input_data.get(subject_mapping["calculus_II"], 0) +
        0.042979 * input_data.get(subject_mapping["embedded_software_exercise"], 0) +
        0.0376 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.026179 * input_data.get(subject_mapping["statistics"], 0) +
        0.025364 * input_data.get(subject_mapping["basic_math"], 0) +
        0.020675 * input_data.get(subject_mapping["information_science_exercise"], 0) +
        0.014685 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.010799 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.005275 * input_data.get(subject_mapping["programming_languages_and_compilers"], 0) +
        0.004498 * input_data.get(subject_mapping["AI_programming"], 0) +
        0.004319 * input_data.get(subject_mapping["discrete_math_III"], 0) +
        0.001764 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.000382 * input_data.get(subject_mapping["career_development"], 0)
    )

  # プログラミング基礎Ⅰ
  def calculate_programming_basic_I(input_data):
    return (
        0.013 +
        0.851749 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.793431 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.644141 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.483545 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.454249 * input_data.get(subject_mapping["career_development"], 0) +
        0.239499 * input_data.get(subject_mapping["basic_math"], 0) +
        0.087042 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.041223 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.00962 * input_data.get(subject_mapping["basic_seminar"], 0)
    )

  # プログラミング基礎Ⅱ
  def calculate_programming_basic_II(input_data):
    return (
        0.599 +
        0.91949 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.671385 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.481069 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.372151 * input_data.get(subject_mapping["information_science_exercise"], 0) +
        0.293646 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.217331 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.158353 * input_data.get(subject_mapping["career_development"], 0) +
        0.127038 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.110298 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.088571 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.082314 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.055617 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.036048 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.029012 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.01201 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.009988 * input_data.get(subject_mapping["basic_math"], 0) +
        0.004472 * input_data.get(subject_mapping["basic_seminar"], 0)
    )

  # モバイルプログラミング演習
  def calculate_mobile_programming_exercise(input_data):
    return (
        -0.324 +
        1.324335 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        1.212487 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.769703 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.282919 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.196161 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.18261 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.070763 * input_data.get(subject_mapping["career_development"], 0) +
        0.04212 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.033432 * input_data.get(subject_mapping["basic_math"], 0)
        )

  # 基礎物理
  def calculate_basic_physics(input_data):
    return (
        -0.202 +
        3.70275 * input_data.get(subject_mapping["programming_intro"], 0) +
        3.462566 * input_data.get(subject_mapping["basic_math"], 0) +
        2.410692 * input_data.get(subject_mapping["basic_seminar"], 0) +
        1.689607 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        1.218807 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.989401 * input_data.get(subject_mapping["career_development"], 0) +
        0.716528 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.202485 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.010779 * input_data.get(subject_mapping["information_literacy"], 0)
    )

  # 微分積分Ⅰ
  def calculate_calculus_I(input_data):
    return (
        -0.111 +
        1.218783 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.585572 * input_data.get(subject_mapping["basic_math"], 0) +
        0.573684 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.549124 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.476039 * input_data.get(subject_mapping["career_development"], 0) +
        0.392858 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.187753 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.071294 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.00179 * input_data.get(subject_mapping["sports_science_exercise"], 0)
    )

  # 微分積分Ⅱ
  def calculate_calculus_II(input_data):
    return (
        0.026 +
        1.160151 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.877652 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.420264 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.278527 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.255912 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.241445 * input_data.get(subject_mapping["basic_math"], 0) +
        0.217549 * input_data.get(subject_mapping["career_development"], 0) +
        0.211688 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.17942 * input_data.get(subject_mapping["information_science_basic_exercise"], 0) +
        0.171264 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.158791 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.131399 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.081555 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.076063 * input_data.get(subject_mapping["mathematical_culture_I"], 0) +
        0.03778 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.035689 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.027349 * input_data.get(subject_mapping["computer_architecture_I"], 0)
    )

  # 数理的教養Ⅰ
  def calculate_mathematical_culture_I(input_data):
    return (
        0.02 +
        0.776533 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.427632 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.390762 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.370972 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.342682 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.322957 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.312719 * input_data.get(subject_mapping["information_science_basic_exercise"], 0) +
        0.3108 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.251784 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.194869 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.116836 * input_data.get(subject_mapping["basic_math"], 0) +
        0.108125 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.09949 * input_data.get(subject_mapping["career_development"], 0) +
        0.084818 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.04201 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.013592 * input_data.get(subject_mapping["information_literacy"], 0)
    )

  # 線形代数Ⅱ
  def calculate_linear_algebra_II(input_data):
    return (
        -0.089 +
        2.08349 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        1.253758 * input_data.get(subject_mapping["basic_math"], 0) +
        0.901302 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.871003 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.6286 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.174305 * input_data.get(subject_mapping["career_development"], 0) +
        0.161565 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.15588 * input_data.get(subject_mapping["information_literacy"], 0)
    )

  # 計算機構成論Ⅰ
  def calculate_computer_architecture_I(input_data):
    return (
        -0.042 +
        1.175108 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        1.153717 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.681188 * input_data.get(subject_mapping["basic_math"], 0) +
        0.30117 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.041714 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.037965 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.035059 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.006656 * input_data.get(subject_mapping["career_development"], 0)
    )

  # 計算機構成論Ⅱ
  def calculate_computer_architecture_II(input_data):
    return (
        -0.107 +
        0.404537 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        0.232634 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.22842 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.218947 * input_data.get(subject_mapping["career_development"], 0) +
        0.184652 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.154322 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.127765 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.100932 * input_data.get(subject_mapping["basic_math"], 0) +
        0.097753 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.096895 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.053488 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.053231 * input_data.get(subject_mapping["information_science_foundation_exercise"], 0) +
        0.043957 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.039608 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.022829 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.009352 * input_data.get(subject_mapping["basic_physics"], 0)
    )

  # 離散数学Ⅱ
  def calculate_discrete_math_II(input_data):
    return (
        0.175 +
        2.157348 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        1.071187 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.715022 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.449792 * input_data.get(subject_mapping["career_development"], 0) +
        0.398879 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.392469 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.092053 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.046097 * input_data.get(subject_mapping["basic_math"], 0)
    )

  # 離散数学Ⅲ
  def calculate_discrete_math_III(input_data):
    return (
        0.175 +
        2.157348 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        1.071187 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.715022 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.449792 * input_data.get(subject_mapping["career_development"], 0) +
        0.398879 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.392469 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.092053 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.046097 * input_data.get(subject_mapping["basic_math"], 0)
    )

  # ＡＩプログラミング演習
  def calculate_ai_programming_exercise(input_data):
    return (
        0.619 +
        1.351449 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        1.03066 * input_data.get(subject_mapping["computer_architecture_I"], 0) +
        1.029847 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.969927 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.794864 * input_data.get(subject_mapping["information_literacy"], 0) +
        0.684514 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.638172 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.393172 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.377879 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.33879 * input_data.get(subject_mapping["information_science_basic_exercise"], 0) +
        0.286723 * input_data.get(subject_mapping["career_development"], 0) +
        0.265879 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.149256 * input_data.get(subject_mapping["basic_math"], 0) +
        0.099576 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.089802 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.01065 * input_data.get(subject_mapping["linear_algebra_I"], 0)
    )

  # 情報理論・確率論
  def calculate_information_theory_and_probability(input_data):
    return (
        0.756 +
        1.241754 * input_data.get(subject_mapping["information_literacy"], 0) +
        1.119312 * input_data.get(subject_mapping["sports_science_exercise"], 0) +
        0.376361 * input_data.get(subject_mapping["programming_intro"], 0) +
        0.335848 * input_data.get(subject_mapping["basic_physics"], 0) +
        0.305905 * input_data.get(subject_mapping["linear_algebra_II"], 0) +
        0.266411 * input_data.get(subject_mapping["basic_math"], 0) +
        0.265963 * input_data.get(subject_mapping["career_development"], 0) +
        0.187989 * input_data.get(subject_mapping["programming_basic_I"], 0) +
        0.184411 * input_data.get(subject_mapping["calculus_I"], 0) +
        0.161099 * input_data.get(subject_mapping["linear_algebra_I"], 0) +
        0.152988 * input_data.get(subject_mapping["discrete_math_II"], 0) +
        0.146652 * input_data.get(subject_mapping["basic_seminar"], 0) +
        0.124931 * input_data.get(subject_mapping["mobile_programming_exercise"], 0) +
        0.056838 * input_data.get(subject_mapping["information_science_basic_exercise"], 0) +
        0.035296 * input_data.get(subject_mapping["discrete_math_I"], 0) +
        0.003903 * input_data.get(subject_mapping["computer_architecture_I"], 0)
    )

  return computer_network, data_structures_and_algorithms_I, data_structures_and_algorithms_I_exercise, hardware_experiment_I, hardware_design_I, hardware_design_II, programming_basic_I, programming_basic_II, mobile_programming_exercise, basic_physics, calculus_I, calculus_II, mathematical_culture_I, linear_algebra_II, computer_architecture_I, computer_architecture_II, discrete_math_II, ai_programming_exercise, information_theory_and_probability

In [ ]:
# 入力
input_data = {
    "コンピューターネットワーク" : -1, "データ構造とアルゴリズムⅠ" : -1, "データ構造とアルゴリズムⅠ演習" : -1, "ハードウェア実験Ⅰ" : -1,
    "ハードウェア設計Ⅰ": -1, "ハードウェア設計Ⅱ" : -1, "プログラミング基礎Ⅰ" : -1, "プログラミング基礎Ⅱ" : -1,
    "モバイルプログラミング演習" : -1, "基礎物理" : -1, "微分積分Ⅰ" : -1, "微分積分Ⅱ" : -1, "数理的教養Ⅰ" : -1, "線形代数Ⅱ" : -1,
    "計算機構成論Ⅰ" : -1, "計算機構成論Ⅱ" : -1, "離散数学Ⅱ" : -1, "離散数学Ⅲ" : -1, "ＡＩプログラミング演習" : -1,
    "情報理論・確率論" : -1
    }

# Gradio


In [ ]:
def greet(network_grade, algo_grade, algo_practice_grade, hardware_exp_grade):
  grade_mapping = {'S': 4, 'A': 3, 'B': 2, 'C': 1, 'D': 0}
  return f"Selected grades:\n- Computer Network: {network_grade}\n- Data Structure and Algorithm I: {algo_grade}\n- Data Structure and Algorithm I Exercise: {algo_practice_grade}\n- Hardware Experiment I: {hardware_exp_grade}"

demo = gr.Interface(
    fn=greet,
    inputs=[
        gr.Radio(["S", "A", "B", "C", "D"], label="コンピュータネットワーク"),
        gr.Radio(["S", "A", "B", "C", "D"], label="データ構造とアルゴリズムⅠ"),
        gr.Radio(["S", "A", "B", "C", "D"], label="データ構造とアルゴリズムⅠ演習"),
        gr.Radio(["S", "A", "B", "C", "D"], label="ハードウェア実験Ⅰ"),
    ],
    outputs=gr.Textbox(),
)

demo.launch()
